In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import seaborn as sns
from arch import arch_model

# Set random seed for reproducibility
np.random.seed(42)

def generate_garch11(omega, alpha, beta, n_points=1000):
    """
    Generate a GARCH(1,1) process:
    r_t = σ_t * ε_t
    σ²_t = ω + α * r²_{t-1} + β * σ²_{t-1}
    
    Parameters:
    -----------
    omega : float
        The constant term in variance equation
    alpha : float
        The coefficient for previous squared return
    beta : float
        The coefficient for previous variance
    n_points : int
        Number of points to generate
    
    Returns:
    --------
    tuple
        (returns, conditional_variances)
    """
    # Initialize arrays
    returns = np.zeros(n_points)
    variances = np.zeros(n_points)
    
    # Set initial variance
    variances[0] = omega / (1 - alpha - beta)  # Unconditional variance
    
    # Generate process
    for t in range(1, n_points):
        # Generate random shock
        z = np.random.standard_normal()
        
        # Calculate return
        returns[t] = np.sqrt(variances[t-1]) * z
        
        # Update variance
        variances[t] = omega + alpha * returns[t-1]**2 + beta * variances[t-1]
    
    return returns, variances

def plot_garch_characteristics(returns, variances):
    """
    Create a comprehensive plot showing key characteristics of GARCH process
    """
    fig, axes = plt.subplots(3, 2, figsize=(15, 12))
    
    # Plot returns
    axes[0, 0].plot(returns)
    axes[0, 0].set_title('Returns')
    axes[0, 0].grid(True)
    
    # Plot conditional volatility
    axes[0, 1].plot(np.sqrt(variances))
    axes[0, 1].set_title('Conditional Volatility (σ_t)')
    axes[0, 1].grid(True)
    
    # Plot squared returns
    axes[1, 0].plot(returns**2)
    axes[1, 0].set_title('Squared Returns')
    axes[1, 0].grid(True)
    
    # Plot conditional variance
    axes[1, 1].plot(variances)
    axes[1, 1].set_title('Conditional Variance (σ²_t)')
    axes[1, 1].grid(True)
    
    # QQ plot of standardized returns
    standardized_returns = returns / np.sqrt(variances)
    stats.probplot(standardized_returns, dist="norm", plot=axes[2, 0])
    axes[2, 0].set_title('Q-Q Plot of Standardized Returns')
    
    # Plot return distribution
    sns.histplot(returns, stat='density', kde=True, ax=axes[2, 1])
    x = np.linspace(min(returns), max(returns), 100)
    axes[2, 1].plot(x, stats.norm.pdf(x, 0, np.std(returns)))
    axes[2, 1].set_title('Return Distribution vs Normal')
    
    plt.tight_layout()
    plt.show()

def compare_volatility_regimes():
    """
    Compare GARCH processes with different persistence levels
    """
    # Generate processes with different persistence (alpha + beta)
    params = [
        (0.01, 0.1, 0.85),  # High persistence
        (0.01, 0.3, 0.45),  # Medium persistence
        (0.01, 0.1, 0.1),   # Low persistence
    ]
    
    fig, axes = plt.subplots(len(params), 2, figsize=(15, 4*len(params)))
    
    for i, (omega, alpha, beta) in enumerate(params):
        returns, variances = generate_garch11(omega, alpha, beta)
        persistence = alpha + beta
        
        # Plot returns
        axes[i, 0].plot(returns)
        axes[i, 0].set_title(f'Returns (persistence = {persistence:.2f})')
        axes[i, 0].grid(True)
        
        # Plot volatility
        axes[i, 1].plot(np.sqrt(variances))
        axes[i, 1].set_title(f'Volatility (persistence = {persistence:.2f})')
        axes[i, 1].grid(True)
    
    plt.tight_layout()
    plt.show()

def analyze_volatility_clustering():
    """
    Demonstrate and analyze volatility clustering in GARCH process
    """
    # Generate GARCH process
    omega, alpha, beta = 0.01, 0.1, 0.85
    returns, variances = generate_garch11(omega, alpha, beta)
    
    # Plot autocorrelation of absolute returns
    fig, axes = plt.subplots(2, 1, figsize=(12, 8))
    
    # Autocorrelation of returns
    lags = 20
    acf_returns = [np.corrcoef(returns[:-k], returns[k:])[0,1] for k in range(1, lags+1)]
    axes[0].bar(range(1, lags+1), acf_returns)
    axes[0].set_title('Autocorrelation of Returns')
    axes[0].grid(True)
    
    # Autocorrelation of absolute returns (volatility clustering)
    acf_abs_returns = [np.corrcoef(np.abs(returns[:-k]), np.abs(returns[k:]))[0,1] 
                      for k in range(1, lags+1)]
    axes[1].bar(range(1, lags+1), acf_abs_returns)
    axes[1].set_title('Autocorrelation of Absolute Returns (Volatility Clustering)')
    axes[1].grid(True)
    
    plt.tight_layout()
    plt.show()

def fit_garch_model():
    """
    Demonstrate GARCH model fitting on simulated data
    """
    # Generate true GARCH process
    true_omega, true_alpha, true_beta = 0.01, 0.1, 0.85
    returns, _ = generate_garch11(true_omega, true_alpha, true_beta)
    
    # Fit GARCH model
    model = arch_model(returns, vol='Garch', p=1, q=1)
    results = model.fit(disp='off')
    
    # Print results
    print("\nGARCH Model Fitting Results:")
    print("True parameters:")
    print(f"ω (omega): {true_omega}")
    print(f"α (alpha): {true_alpha}")
    print(f"β (beta): {true_beta}")
    print("\nEstimated parameters:")
    print(results.summary().tables[1])
    
    # Plot true vs fitted volatility
    plt.figure(figsize=(12, 6))
    plt.plot(np.sqrt(results.conditional_volatility**2), 
             label='Fitted Volatility')
    plt.title('GARCH(1,1) Fitted Conditional Volatility')
    plt.legend()
    plt.grid(True)
    plt.show()

# Run analyses
print("Generating and analyzing GARCH process...")
returns, variances = generate_garch11(0.01, 0.1, 0.85)
plot_garch_characteristics(returns, variances)

print("\nComparing different volatility regimes...")
compare_volatility_regimes()

print("\nAnalyzing volatility clustering...")
analyze_volatility_clustering()

print("\nDemonstrating GARCH model fitting...")
fit_garch_model()